In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import ttest_ind
from dipy.core.sphere import disperse_charges, HemiSphere
from dipy.core.gradients import gradient_table
from dipy.data import get_sphere
from dipy.sims.voxel import multi_tensor, multi_tensor_odf, single_tensor_odf
from dipy.viz import window, actor
from dipy.reconst.csdeconv import mask_for_response_ssst, response_from_mask_ssst
import nibabel as nib

#SCRIPT

import nibabel as nib
import numpy as np
from numpy.linalg import norm
from scipy import stats
from math import *
import sys, glob, xlwt, os.path, re

In [7]:
# Open the file in read mode
with open('real_data/hardi-scheme.bvec.txt', 'r') as file:
    # Read the lines from the file
    lines = file.readlines()

# Initialize an empty matrix to store the values
real_bvecs = []

# Iterate through each line
for line in lines:
    # Split the line into individual values
    values = line.split() 
    # Convert the values to integers and append to the matrix
    real_bvecs.append([float(value) for value in values])

# Convert the list of lists into a 2D NumPy array
real_bvecs = np.array(real_bvecs).T


# Open the file in read mode
with open('real_data/hardi-scheme.bval.txt', 'r') as file:
    # Read the lines from the file
    lines = file.readlines()

# Initialize an empty matrix to store the values
real_bvals = []

# Iterate through each line
for line in lines:
    # Split the line into individual values
    values = line.split() 
    # Convert the values to integers and append to the matrix
    real_bvals.append([float(value) for value in values])

# Convert the list of lists into a 2D NumPy array
real_bvals = np.array(real_bvals).reshape(-1)

gtab = gradient_table(real_bvals, real_bvecs)

mask = nib.load('real_data/mask.nii').get_fdata().astype(bool)
white_mask = nib.load('real_data/white_matter_mask.nii').get_fdata().astype(bool)

hardi_snr10 = nib.load('real_data/DWIS_hardi-scheme_SNR-10.nii').get_fdata()
hardi_snr20 = nib.load('real_data/DWIS_hardi-scheme_SNR-20.nii').get_fdata()
hardi_snr30 = nib.load('real_data/DWIS_hardi-scheme_SNR-30.nii').get_fdata()

hardi_snr10_masked = hardi_snr10[mask]
hardi_snr20_masked = hardi_snr20[mask]
hardi_snr30_masked = hardi_snr30[mask]
hardi_snr10_masked *= (100/hardi_snr10_masked[0][0])
hardi_snr20_masked *= hardi_snr20_masked * (100/hardi_snr20_masked[0][0])
hardi_snr30_masked *= hardi_snr30_masked * (100/hardi_snr30_masked[0][0])

hardi_snr10_white_masked = hardi_snr10[white_mask]
hardi_snr20_white_masked = hardi_snr20[white_mask]
hardi_snr30_white_masked = hardi_snr30[white_mask]
hardi_snr10_white_masked *= (100/hardi_snr10_white_masked[0][0])
hardi_snr20_white_masked *= (100/hardi_snr20_white_masked[0][0])
hardi_snr30_white_masked *= (100/hardi_snr30_white_masked[0][0])

hardi_white_masked_all = np.vstack((hardi_snr10_white_masked, hardi_snr20_white_masked, hardi_snr30_white_masked))
np.random.seed(42)
np.random.shuffle(hardi_white_masked_all)

S = np.load("synthetic_data/S.npy")
S_noiseless = np.load("synthetic_data/S_noiseless.npy")
F = np.load('synthetic_data/F.npy')
angle_pairs = pd.read_pickle("synthetic_data/angle_pairs.pkl")

In [19]:
sphere = get_sphere('symmetric724')
response_mask = mask_for_response_ssst(gtab, hardi_snr30, roi_radii=20, fa_thr=0.7)
response, ratio = response_from_mask_ssst(gtab, hardi_snr30, response_mask)
print(response)
print(ratio)
scene = window.Scene()
evals = response[0]
evecs = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]).T

response_odf = single_tensor_odf(sphere.vertices, evals, evecs)
# transform our data from 1D to 4D
response_odf = response_odf[None, None, None, :]
response_actor = actor.odf_slicer(response_odf, sphere=sphere,
                                  colormap='plasma')
scene.add(response_actor)
window.show(scene)
scene.rm(response_actor)

(array([0.00164641, 0.00038227, 0.00038227]), 32767.0)
0.23218691030646116


UNDERSTANDING THE "compute_local_metrics.py" SCRIPT

In [24]:
### load ground-truth directions
################################
print("-> opening ground-truth...")

niiGT = nib.load('real_data/ground-truth-peaks.nii.gz')
niiGT_hdr = niiGT.header
niiGT_img = niiGT.get_fdata()

niiGT_dim = niiGT_hdr.get_data_shape()

nx = niiGT_dim[0]
ny = niiGT_dim[1]
nz = niiGT_dim[2]

print("[OK]\n")

-> opening ground-truth...
[OK]


In [29]:
### load mask of interest
#########################
print("-> opening binary mask of interest..."),

niiMASK = nib.load('real_data/white_matter_mask.nii.gz')
niiMASK_hdr = niiMASK.header
niiMASK_img = niiMASK.get_fdata() # 0-1 mask

niiMASK_dim = niiMASK_hdr.get_data_shape()

niiMASK_idx = niiMASK_img==1 #Boolean mask

print("[OK]\n")

-> opening binary mask of interest...


In [1]:
import pandas as pd

In [2]:
angle_pairs = pd.read_pickle("synthetic_data/angle_pairs.pkl")

In [3]:
print(angle_pairs)

[(86.35289061629949, 164.67971184861608), (78.26037341234078, 157.42585221843626), (20.86621192637322, -28.948069478035848), (52.809562048566825, -8.955654047064344), (76.77581076817995, -64.39586382077414), (57.15381058153167, -53.32792488737147), (54.73987552935679, -35.6981081580442), (63.59812437750721, -5.407817724097641), (86.10474540623034, 117.39916318744284), (51.34239816612954, -158.21479284164442), (86.95117323911015, 88.59829520428002), (62.2309196004447, -172.0368820739597), (73.14844129342923, -103.1373924820191), (88.42257643749772, -137.03600746959495), (55.65948212908443, 4.944921121159132), (71.54112382571455, -164.55421204907762), (70.24571679614154, -114.2984983152024), (88.17676902273881, 65.28930087534702), (64.52182803545436, -96.36690282688276), (68.4625811214182, -125.3228241761383), (68.48501320908652, 63.36144896146067), (82.54424341245328, -163.4287394179697), (60.685447716563885, 163.73343610148962), (89.00505925821409, 175.33097260558378), (71.075677956777

In [9]:
def spherical_to_cartesian(theta, phi):
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    return x, y, z

In [13]:
import numpy as np
from math import *
angle_matrix = []

for i in range(len(angle_pairs)):
    angles = []
    for j in range(len(angle_pairs)): 
        if i == j:
            continue
        x_i, y_i, z_i = spherical_to_cartesian(angle_pairs[i][0], angle_pairs[i][1])
        x_j, y_j, z_j = spherical_to_cartesian(angle_pairs[j][0], angle_pairs[j][1])
        
        angles.append(acos(min(1.0, abs(x_i * x_j + y_i * y_j + z_i * z_j))) / pi * 180)
    
    angle_matrix.append(angles)

angle_matrix = np.array(angle_matrix)

In [18]:
angle_matrix.min(axis = 1)

array([ 4.52523629,  4.71054596,  6.16539295,  9.08109231,  2.6617341 ,
        8.29687196,  2.57903745,  4.5687461 ,  6.48096008,  4.57005777,
        4.3754504 ,  7.74722726,  7.99526939,  6.53006897,  3.09382745,
        1.64783412,  4.57005777,  3.77792204,  9.64942618,  6.587505  ,
        6.587505  , 12.92606571, 10.82287719,  1.37137574,  7.81960513,
        2.75650408,  2.47643197,  5.46771965,  5.06471301,  7.81960513,
        8.87794989,  8.71045219,  2.45077819,  8.09613305,  4.71054596,
       10.59572412,  6.03811922, 10.49076038,  4.52523629,  5.92311368,
        4.83020199,  5.42249568,  1.13589206,  2.48758189,  1.64783412,
        4.71314191,  2.6617341 ,  5.53753138,  5.60840291,  0.96416224,
        6.17004924,  1.63121002,  2.44698875,  8.99949152,  1.23798728,
        7.67613418,  6.23311145,  6.78012078,  4.49662078,  5.53753138,
        8.20984412,  3.3024832 ,  3.2204864 ,  6.49862095, 10.24439234,
        0.90150845,  2.56501409,  2.48758189,  1.07388823,  2.39

In [16]:
angle_matrix

array([[83.31978873, 69.52387434, 66.06254637, ..., 86.47282222,
        44.58853538, 88.3715868 ],
       [83.31978873, 73.4857705 , 50.1585876 , ..., 17.83545093,
        60.67784738, 76.01284977],
       [69.52387434, 73.4857705 , 55.44730651, ..., 63.6705378 ,
        75.2586769 , 22.12163464],
       ...,
       [86.47282222, 17.83545093, 63.6705378 , ..., 65.39992004,
        45.44241769, 61.02021433],
       [44.58853538, 60.67784738, 75.2586769 , ..., 37.09217594,
        45.44241769, 56.59778678],
       [88.3715868 , 76.01284977, 22.12163464, ..., 28.41857363,
        61.02021433, 56.59778678]])

In [19]:
import numpy as np
from dipy.core.sphere import disperse_charges, HemiSphere, Sphere
from dipy.core.gradients import gradient_table
from dipy.data import get_sphere
from dipy.sims.voxel import multi_tensor, multi_tensor_odf
from dipy.direction import peak_directions
import pandas as pd

thetas = list(np.load("synthetic_data/thetas.npy"))
phis = list(np.load("synthetic_data/phis.npy"))

hemisphere = HemiSphere(theta=thetas, phi=phis) # We already dispersed charges when building the hemisphere in dipy_test
sphere = Sphere(xyz=np.vstack((hemisphere.vertices, -hemisphere.vertices)))

F = np.load("synthetic_data/F.npy")

def detect_peaks(F, relative_peak_threshold, min_separation_angle):
    peak_format = np.zeros((len(F), 15))
    for i, sample in enumerate(F):
        # Duplicate the sample for both hemispheres
        F_sphere = np.hstack((sample, sample)) / 2

        # Find peak directions
        directions, values, indices = peak_directions(F_sphere, sphere, relative_peak_threshold, min_separation_angle)
        directions_flattened = directions.flatten()
        peak_format[i][0:len(directions_flattened)] = directions_flattened
    return peak_format


In [32]:
non_zero = np.nonzero(F[0])
for i in non_zero[0]:
    print(i)
    print(spherical_to_cartesian(thetas[i], phis[i]))

87
(-0.37890021440593835, -0.5761459231473628, 0.7242171654716608)
176
(-0.8811615911315133, -0.27118010236205414, -0.38731847670552155)


In [25]:
peaks_f_0 = detect_peaks(F, 0.1, 15)

In [26]:
peaks_f_0[0]

array([ 0.88116159,  0.2711801 ,  0.38731848, -0.37890021, -0.57614592,
        0.72421717,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [17]:
a = pd.read_pickle("real_data/ground_truth_peaks_white_masked.pkl")

In [57]:
num_rows_to_sample = 1

# Generate random indices to sample
random_indices = np.random.choice(a.shape[0], size=num_rows_to_sample, replace=False)

# Use the random indices to extract the sampled rows
sampled_rows = a[[1955], :]

pd.to_pickle(sampled_rows, "small_peaks1.pkl")

In [52]:
import nibabel as nib
niiGT = nib.load('real_data/ground-truth-peaks.nii.gz')
niiGT_img = niiGT.get_fdata()

niiMASK = nib.load('real_data/white_matter_mask.nii.gz')
niiMASK_img = niiMASK.get_fdata().astype(bool)

In [53]:
masked_GT = niiGT_img[niiMASK_img]

In [54]:
nonzero_counts = np.count_nonzero(masked_GT, axis=1) / 3

In [56]:
np.where(nonzero_counts == 3)

(array([ 1955,  1975,  2359,  2377,  2528,  2660,  2692,  2793,  2943,
         2949,  2950,  2988,  2994,  2995,  3033,  3039,  3040,  3192,
         3205,  3207,  3460,  3465,  3466,  3507,  3512,  3513,  3554,
         3559,  3560,  3744,  3748,  3760,  3766,  4073,  4077,  4078,
         4119,  4120,  4124,  4125,  4166,  4167,  4171,  4172,  4214,
         4219,  4323,  4353,  4597,  4700,  4701,  4704,  4705,  4706,
         4747,  4748,  4751,  4752,  4753,  4794,  4795,  4798,  4799,
         4800,  4846,  5216,  5302,  5303,  5306,  5307,  5349,  5350,
         5352,  5353,  5354,  5398,  5399,  5401,  5402,  5403,  5451,
         5835,  5876,  5877,  5879,  5880,  5882,  5883,  5924,  5925,
         5926,  5928,  5929,  5931,  5932,  5974,  5975,  5977,  5978,
         5980,  5981,  6026,  6028,  6123,  6135,  6137,  6151,  6152,
         6168,  6169,  6170,  6171,  6188,  6576,  6577,  6586,  6587,
         6588,  6589,  6590,  6605,  6606,  6607,  6608,  6609,  6610,
      

In [79]:
pd.to_pickle(masked_GT, 'real_data/ground_truth_peaks_white_masked.pkl')